In [1]:
%run functions.ipynb

In [2]:
# 1. Get/read data;
df = pd.read_csv('data/METAR_2014_2023.csv')[-24*90:]
df['date'] = pd.to_datetime(df['date'])
print(df.date.min(), df.date.max())
print(df.shape)
df[:2]

2023-10-10 00:00:00 2023-12-31 00:00:00
(2160, 3)


,airport_id,date,metar
87430,KMIA,2023-10-10,KMIA 101310Z 00000KT 10SM FEW018 BKN250 27/23 A2991 RMK AO2 T02720228
87431,KMIA,2023-10-10,KMIA 101353Z 00000KT 10SM FEW017 BKN250 28/23 A2992 RMK AO2 SLP132 T02830228


In [3]:
# 2. Preprocess the data by parsing METAR strings;
df1 = get_preprocessing(df)

# add fake row - to get new hour prediction, (not prediction for the last hour in the dataset)
df1 = pd.concat([df1, df1[-1:]])
df1['time'] = pd.date_range(start = df1.index[0], periods = df1.shape[0], freq = 'h')
df1.set_index('time', inplace = True)

print(df1.shape)
df1[-2:]

(1980, 38)


,wind_dir,wind_speed,wind_gust,vis,temp,dewpt,press,sky,vis_unclear_flg,wind_speed_flg,wind_gust_flg,wind_dir_flg,weather_rain,weather_rain_flg,weather_ts,weather_fog,weather_fog_flg,sky_cnt_BKN,sky_cnt_CB,sky_cnt_CLR,sky_cnt_FEW,sky_cnt_OVC,sky_cnt_SCT,sky_cnt_TCU,sky_flg_BKN,sky_flg_FEW,sky_flg_SCT,sky_flg_OVC,sky_flg_CLR,sky_flg_CB,sky_flg_TCU,sky_avg_BKN,sky_avg_CB,sky_avg_CLR,sky_avg_FEW,sky_avg_OVC,sky_avg_SCT,sky_avg_TCU
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-12-31 23:00:00,290.00,0.00,17.00,10.00,18.00,9.00,30.22,"[(CLR, None, None)]",0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0,0,0,1,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2024-01-01 00:00:00,290.00,0.00,17.00,10.00,18.00,9.00,30.22,"[(CLR, None, None)]",0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0,0,0,1,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [4]:
# 3. Create features;
n = 10
feats = [f for f in list(df1.columns) if f != 'sky']
df_feats = get_features(df1, feats = feats)[-n:].reset_index(drop = True)

# create xgb matrix
feats_target = list(df1.columns)
feats_used = [f for f in df_feats.columns if f not in feats_target + ['time']]
x = xgb.DMatrix(df_feats[feats_used].values, 
                label = None, 
                feature_names = feats_used) 

print(df_feats.shape)
df_feats[-2:]

(10, 262)


,time,wind_dir,wind_dir_lag1,wind_dir_lag24,wind_dir_RM_lag1_WS3,wind_dir_S_RM_lag1_SL24_WS3,wind_dir_lag1to3,wind_dir_lag24to72,wind_speed,wind_speed_lag1,wind_speed_lag24,wind_speed_RM_lag1_WS3,wind_speed_S_RM_lag1_SL24_WS3,wind_speed_lag1to3,wind_speed_lag24to72,wind_gust,wind_gust_lag1,wind_gust_lag24,wind_gust_RM_lag1_WS3,wind_gust_S_RM_lag1_SL24_WS3,wind_gust_lag1to3,wind_gust_lag24to72,vis,vis_lag1,vis_lag24,vis_RM_lag1_WS3,vis_S_RM_lag1_SL24_WS3,vis_lag1to3,vis_lag24to72,temp,temp_lag1,temp_lag24,temp_RM_lag1_WS3,temp_S_RM_lag1_SL24_WS3,temp_lag1to3,temp_lag24to72,dewpt,dewpt_lag1,dewpt_lag24,dewpt_RM_lag1_WS3,dewpt_S_RM_lag1_SL24_WS3,dewpt_lag1to3,dewpt_lag24to72,press,press_lag1,press_lag24,press_RM_lag1_WS3,press_S_RM_lag1_SL24_WS3,press_lag1to3,press_lag24to72,vis_unclear_flg,vis_unclear_flg_lag1,vis_unclear_flg_lag24,vis_unclear_flg_RM_lag1_WS3,vis_unclear_flg_S_RM_lag1_SL24_WS3,vis_unclear_flg_lag1to3,vis_unclear_flg_lag24to72,wind_speed_flg,wind_speed_flg_lag1,wind_speed_flg_lag24,wind_speed_flg_RM_lag1_WS3,wind_speed_flg_S_RM_lag1_SL24_WS3,wind_speed_flg_lag1to3,wind_speed_flg_lag24to72,wind_gust_flg,wind_gust_flg_lag1,wind_gust_flg_lag24,wind_gust_flg_RM_lag1_WS3,wind_gust_flg_S_RM_lag1_SL24_WS3,wind_gust_flg_lag1to3,wind_gust_flg_lag24to72,wind_dir_flg,wind_dir_flg_lag1,wind_dir_flg_lag24,wind_dir_flg_RM_lag1_WS3,...,sky_flg_OVC_S_RM_lag1_SL24_WS3,sky_flg_OVC_lag1to3,sky_flg_OVC_lag24to72,sky_flg_CLR,sky_flg_CLR_lag1,sky_flg_CLR_lag24,sky_flg_CLR_RM_lag1_WS3,sky_flg_CLR_S_RM_lag1_SL24_WS3,sky_flg_CLR_lag1to3,sky_flg_CLR_lag24to72,sky_flg_CB,sky_flg_CB_lag1,sky_flg_CB_lag24,sky_flg_CB_RM_lag1_WS3,sky_flg_CB_S_RM_lag1_SL24_WS3,sky_flg_CB_lag1to3,sky_flg_CB_lag24to72,sky_flg_TCU,sky_flg_TCU_lag1,sky_flg_TCU_lag24,sky_flg_TCU_RM_lag1_WS3,sky_flg_TCU_S_RM_lag1_SL24_WS3,sky_flg_TCU_lag1to3,sky_flg_TCU_lag24to72,sky_avg_BKN,sky_avg_BKN_lag1,sky_avg_BKN_lag24,sky_avg_BKN_RM_lag1_WS3,sky_avg_BKN_S_RM_lag1_SL24_WS3,sky_avg_BKN_lag1to3,sky_avg_BKN_lag24to72,sky_avg_CB,sky_avg_CB_lag1,sky_avg_CB_lag24,sky_avg_CB_RM_lag1_WS3,sky_avg_CB_S_RM_lag1_SL24_WS3,sky_avg_CB_lag1to3,sky_avg_CB_lag24to72,sky_avg_CLR,sky_avg_CLR_lag1,sky_avg_CLR_lag24,sky_avg_CLR_RM_lag1_WS3,sky_avg_CLR_S_RM_lag1_SL24_WS3,sky_avg_CLR_lag1to3,sky_avg_CLR_lag24to72,sky_avg_FEW,sky_avg_FEW_lag1,sky_avg_FEW_lag24,sky_avg_FEW_RM_lag1_WS3,sky_avg_FEW_S_RM_lag1_SL24_WS3,sky_avg_FEW_lag1to3,sky_avg_FEW_lag24to72,sky_avg_OVC,sky_avg_OVC_lag1,sky_avg_OVC_lag24,sky_avg_OVC_RM_lag1_WS3,sky_avg_OVC_S_RM_lag1_SL24_WS3,sky_avg_OVC_lag1to3,sky_avg_OVC_lag24to72,sky_avg_SCT,sky_avg_SCT_lag1,sky_avg_SCT_lag24,sky_avg_SCT_RM_lag1_WS3,sky_avg_SCT_S_RM_lag1_SL24_WS3,sky_avg_SCT_lag1to3,sky_avg_SCT_lag24to72,sky_avg_TCU,sky_avg_TCU_lag1,sky_avg_TCU_lag24,sky_avg_TCU_RM_lag1_WS3,sky_avg_TCU_S_RM_lag1_SL24_WS3,sky_avg_TCU_lag1to3,sky_avg_TCU_lag24to72,dt_month,dt_hour
8,2023-12-31 23:00:00,290.00,290.00,180.00,296.67,276.67,0.98,0.65,0.00,4.00,4.00,4.33,7.00,0.92,0.57,17.00,17.00,17.00,17.00,17.00,1.00,1.00,10.00,10.00,10.00,10.00,10.00,1.00,1.00,18.00,19.00,17.00,20.33,18.67,0.93,0.91,9.00,9.00,9.00,8.67,10.00,1.04,0.90,30.22,30.21,30.17,30.20,30.14,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.33,0.00,0.00,0.00,1.00,1.00,1.00,...,0.33,0.00,0.00,1.00,1.00,0.00,0.33,0.33,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"22,000.00",0.00,"7,666.67",0.00,2.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,666.67","2,666.67",0.00,0.00,0.00,0.00,0.00,0.00,"8,333.33",0.00,0.00,0.00,0.00,0.00,0.00,"6,666.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12,23
9,2024-01-01 00:00:00,290.00,290.00,180.00,290.00,240.00,1.00,0.75,0.00,0.00,0.00,2.67,3.00,0.00,0.00,17.00,17.00,17.00,17.00,17.00,1.00,1.00,10.00,10.00,10.00,10.00,10.00,1.00,1.00,18.00,18.00,16.00,19.33,18.00,0.93,0.89,9.00,9.00,10.00,8.67,10.33,1.04,0.97,30.22,30.22,30.19,30.21,30.16,1.

In [5]:
# 4. Make predictions for individual components according the table below;
df_preds = pd.DataFrame()

# make predictions by the models
list_models = os.listdir('models')
for feat_target in feats_target:
    if f'{feat_target}.pkl' in list_models:
        model_xgb = xgb.Booster(model_file = f'models/{feat_target}.pkl')
        df_preds[feat_target] = model_xgb.predict(x)
    else:
        df_preds[feat_target] = np.nan
df_preds['time'] = df_feats.time

# last value predictions
for feat_target in ['dewpt','weather_rain_flg','sky_flg_TCU']:
    df_preds[feat_target] = df_feats[f'{feat_target}_lag1']
df_preds['sky_cnt_TCU'] = df_preds['sky_flg_TCU']

# correct categorical predictions
df_preds['sky'] = df1.sky.values[-df_preds.shape[0]:]

df_preds['wind_speed_flg'] = (df_preds['wind_speed_flg'] > 0.6437).astype(int)
df_preds['wind_dir_flg']   = (df_preds['wind_dir_flg']   > 0.3085).astype(int)
df_preds['wind_gust_flg']  = (df_preds['wind_gust_flg']  > 0.6566).astype(int)
df_preds['vis_unclear_flg'] = (df_preds['vis_unclear_flg'] > 0.2513).astype(int)
df_preds['weather_fog_flg'] = (df_preds['weather_fog_flg'] > 0.2146).astype(int)
df_preds['weather_ts'] = (df_preds['weather_ts'] > 0.3087).astype(int)
df_preds['sky_flg_BKN'] = (df_preds['sky_flg_BKN'] > 0.5372).astype(int)
df_preds['sky_flg_FEW'] = (df_preds['sky_flg_FEW'] > 0.5279).astype(int)
df_preds['sky_flg_SCT'] = (df_preds['sky_flg_SCT'] > 0.5361).astype(int)
df_preds['sky_flg_OVC'] = (df_preds['sky_flg_OVC'] > 0.3971).astype(int)
df_preds['sky_flg_CLR'] = (df_preds['sky_flg_CLR'] > 0.3039).astype(int)
df_preds['sky_flg_CB']  = (df_preds['sky_flg_CB']  > 0.2848).astype(int)

# correct numerical predictions
df_preds.loc[(df_preds['wind_speed_flg'] == 0), 'wind_speed'] = 0
df_preds.loc[(df_preds['wind_speed_flg'] == 0), 'wind_gust'] = 0
df_preds.loc[(df_preds['wind_speed_flg'] == 0), 'wind_dir'] = 0
df_preds.loc[(df_preds['wind_gust_flg'] == 0), 'wind_gust'] = 0
df_preds.loc[(df_preds['wind_dir_flg'] == 0),  'wind_dir']  = 0
              
df_preds.loc[(df_preds['vis_unclear_flg'] == 0), 'vis'] = 10
df_preds.loc[(df_preds['weather_rain_flg'] == 0), 'weather_rain'] = 0
df_preds.loc[(df_preds['weather_fog_flg'] == 0),  'weather_fog']  = 0
df_preds.loc[(df_preds['sky_flg_BKN'] == 0), 'sky_cnt_BKN'] = 0
df_preds.loc[(df_preds['sky_flg_FEW'] == 0), 'sky_cnt_FEW'] = 0
df_preds.loc[(df_preds['sky_flg_SCT'] == 0), 'sky_cnt_SCT'] = 0
df_preds.loc[(df_preds['sky_flg_OVC'] == 0), 'sky_cnt_OVC'] = 0
df_preds.loc[(df_preds['sky_flg_CLR'] == 0), 'sky_cnt_CLR'] = 0
df_preds.loc[(df_preds['sky_flg_CB'] == 0),  'sky_cnt_CB']  = 0

print(df_preds.shape)
df_preds[-2:]

(10, 39)


,wind_dir,wind_speed,wind_gust,vis,temp,dewpt,press,sky,vis_unclear_flg,wind_speed_flg,wind_gust_flg,wind_dir_flg,weather_rain,weather_rain_flg,weather_ts,weather_fog,weather_fog_flg,sky_cnt_BKN,sky_cnt_CB,sky_cnt_CLR,sky_cnt_FEW,sky_cnt_OVC,sky_cnt_SCT,sky_cnt_TCU,sky_flg_BKN,sky_flg_FEW,sky_flg_SCT,sky_flg_OVC,sky_flg_CLR,sky_flg_CB,sky_flg_TCU,sky_avg_BKN,sky_avg_CB,sky_avg_CLR,sky_avg_FEW,sky_avg_OVC,sky_avg_SCT,sky_avg_TCU,time
8,303.05,4.08,0.00,10.00,18.36,9.00,30.22,"[(CLR, None, None)]",0,1,0,1,0.00,0.00,0,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0,0,0,1,0,0.00,"12,131.56","2,719.22",NaN,"5,636.62","11,955.16","10,009.09","2,768.49",2023-12-31 23:00:00
9,0.00,0.00,0.00,10.00,17.22,9.00,30.23,"[(CLR, None, None)]",0,0,0,1,0.00,0.00,0,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0,0,0,1,0,0.00,"11,361.05","2,752.88",NaN,"5,763.58","11,069.74","16,754.45","2,332.82",2024-01-01 00:00:00


In [6]:
# 5. Convert predictions to METAR string
# RULES HERE: https://metar-taf.com/explanation

# get values
def conv_val(x):
    try:
        return x.value()
    except:
        return np.nan

conv1 = lambda x: '0'+str(round(x)) if round(x) < 10 else str(round(x))
def conv2(x):
    x = int(round(x/10) * 10)
    if x < 10:
        return 'VRB'
    if x < 100:
        return f'0{x}'
    return str(x)

def get_str_wind(wind_speed, wind_dir, wind_gust):    
    if wind_speed == 0:
        return '00000KT'
    out_dir   = 'VRB' if wind_dir == 0 else conv2(wind_dir)
    out_speed = conv1(wind_speed)
    out_gust  = '' if wind_gust == 0 else 'G'+conv1(wind_gust) 
    return f'{out_dir}{out_speed}{out_gust}KT'
    
def get_str_vis(x):
    if x < 1/16:
        return '1/16SM'
    if x < 1/8:
        return '1/8SM'
    if x < 1/4:
        return '1/4SM'
    if x < 1/2:
        return '1/2SM'
    if x < 3/4:
        return '3/4SM'
    if x < 1:
        return '1SM'
    if x < 5/4:
        return '1 1/4SM'
    if x < 3/2:
        return '1 1/2SM'
    if x < 7/4:
        return '1 3/4SM'
    if x < 2:
        return '2SM'
    if x < 3:
        return '3SM'
    if x < 4:
        return '4SM'
    if x < 5:
        return '5SM'
    if x < 6:
        return '6SM'
    if x < 7:
        return '7SM'
    if x < 8:
        return '8SM'
    if x < 9:
        return '9SM'   
    return '10SM'

def get_str_temp(temp, dewpt):
    temp  = conv1(temp)
    dewpt = conv1(dewpt)
    return f'{temp}/{dewpt}'

def get_str_press(press):
    return 'A' + str(press)[:5].replace('.','')

def get_str_weather(rain, ts, fog):
    rain = round(rain)
    fog = round(fog)

    if rain in [0,2]:
        out0 = ''
    elif rain == 1:
        out0 = '-'
    else:
        out0 = '+'

    if ts == 1:
        if rain > 0:
            out1 = 'TSRA'
        else:
            out1 = 'TS'
    else:
        out1 = ''

    if fog == 1:
        out2 = 'BR'
    elif fog == 2:
        out2 = 'HZ'
    elif fog == 3:
        out2 = 'FG'
    elif fog > 3:
        out2 = 'FU'
    else:
        out2 = ''
    
    return f'{out0}{out1} {out2}'.strip()

# sky...
# todo - add prediction logic
def get_str_sky(x):
    
    # convert clouds to string from table
    def cloud_conv(df):
        def conv3(x):
            x = int(round(x/100))
            if x < 10:
                return f'00{x}'
            if x < 100:
                return f'0{x}'
            return str(x)
        def get_str3(x):
            return x.cloud + (conv3(x.height) if x.cloud != 'CLR' else '') + x.cb_tcu
        df['metar'] = df.fillna('').apply(get_str3, axis = 1)
        return ' '.join(df.metar)
    
    # pred
    df_out = pd.DataFrame({
        'cloud': ['BKN','FEW','SCT','OVC','CLR','CB','TCU'],
        'cnt': [x.sky_cnt_BKN, x.sky_cnt_FEW, x.sky_cnt_SCT, x.sky_cnt_OVC, x.sky_cnt_CLR, x.sky_cnt_CB, x.sky_cnt_TCU],
        'avg': [x.sky_avg_BKN, x.sky_avg_FEW, x.sky_avg_SCT, x.sky_avg_OVC, x.sky_avg_CLR, x.sky_avg_CB, x.sky_avg_TCU],
    })
    df_out['cnt'] = df_out.cnt.round()
    df_out['avg'] = df_out.avg.fillna(0).map(lambda x: int(round(x/100) * 100))
    df_out = df_out[df_out.cnt >= 1]
    
    # real previous
    df_sky = pd.DataFrame(x.sky)
    if len(df_sky) > 0:
        df_sky[1] = df_sky[1].map(conv_val).fillna(0)
    df_sky.columns = ['cloud','height','cb_tcu']
    df_sky = df_sky.fillna('')
    out = cloud_conv(df_sky)
    
    return out

def get_metar(x):
    s1 = str(x.time)
    out0 = 'KMIA '
    out1 = s1[8:13].replace(' ','') + '53Z '
    return (out0 + out1 + 
            get_str_wind(x.wind_speed, x.wind_dir, x.wind_gust) + ' ' +
            get_str_vis(x.vis) + ' ' +
            get_str_weather(x.weather_rain, x.weather_ts, x.weather_fog) + ' ' +
            get_str_sky(x) + ' ' +
            get_str_temp(x.temp, x.dewpt) + ' ' +
            get_str_press(x.press)
    )
df_preds.apply(get_metar, axis = 1)

0       KMIA 311553Z 25005KT 10SM  CLR 20/08 A3026
1       KMIA 311653Z 26005KT 10SM  CLR 21/08 A3024
2    KMIA 311753Z 25004KT 10SM  FEW040 22/08 A3023
3       KMIA 311853Z 32006KT 10SM  CLR 22/09 A3019
4    KMIA 311953Z 32004KT 10SM  FEW250 21/08 A3019
5    KMIA 312053Z 29005KT 10SM  FEW250 21/08 A3019
6    KMIA 312153Z 31004KT 10SM  FEW250 20/09 A3019
7       KMIA 312253Z 30004KT 10SM  CLR 20/08 A3020
8       KMIA 312353Z 30004KT 10SM  CLR 18/09 A3021
9       KMIA 010053Z 00000KT 10SM  CLR 17/09 A3022
dtype: object